This notebook experiments with FID (Fretcher inception distance) metrics

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data as Data

import torchvision.models as models
from torchvision import transforms

from lib.network import InceptionV3
from lib.metrics import gene

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image